### notebook purpose
- following issue #11:  https://github.com/jasonasher/dc_doh_hackathon/issues/11
extract data from csv and write a new csv



### Notes
- Should probably use a real index to the dataframe we generate, on (year, week, category, censusblock), but here we just use a rownumber index

## Note: To use this for real, you need to add the geopandas code to convert lat/long to census_block_2010



- mhisted@gmail.com  171024

In [3]:
import pandas as pd
import datetime

In [7]:
# change these for the different input files
fName = 'Basic_Business_License_in_2015.csv'
outName = fName.replace('.csv', '-features.csv')


In [5]:
df = pd.read_csv(fName)

In [9]:
# parse the datestrings into useful columns
invalIx = df.LICENSE_EXPIRATION_DATE.apply(lambda x: type(x)!=str)
df = df.loc[~invalIx,:]
df['expDatetime'] = df.LICENSE_EXPIRATION_DATE.apply(lambda x: pd.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.000Z'))
df['expYear'] = df.expDatetime.apply(lambda x: int(x.strftime('%Y')))
df['expWeek'] = df.expDatetime.apply(lambda x: int(x.strftime('%U')))

#print(df.expYear[0:5])
#print(df.expWeek[0:5])

df['issueDatetime'] = df.LICENSE_ISSUE_DATE.apply(lambda x: pd.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.000Z'))
df['issueYear'] = df.issueDatetime.apply(lambda x: int(x.strftime('%Y')))
df['issueWeek'] = df.issueDatetime.apply(lambda x: int(x.strftime('%U')))

#print(df.issueYear[0:10])
#print(np.unique(df.issueYear))
#print(df.columns)



In [11]:
# add code here to convert lat/long into census block numbers

# should look something like this
#df['census_block_2010'] = df[['LATITUDE','LONGITUDE']].\
#    apply(lambda x: fancy_geopandas_function(x[0],x[1]))

# in the meantime - this gives us 149 unique locs
df['census_block_2010'] = np.floor(df.LATITUDE*df.LONGITUDE*10)
print(len(np.unique(df.census_block_2010)))


149


In [12]:
# extract unique values of columns to iterate over
all_types = np.unique(df.LICENSECATEGORY)
all_blocks = np.unique(df.census_block_2010)
all_years = np.unique(df.issueYear)

# subset by licensecategory for speed
dfD = {}
for tT in all_types:
    dfD[tT] = df.loc[df.LICENSECATEGORY == tT,:]

In [13]:
# iterate over year, week, types, census blocks; calculate value, make a list of dicts for convert to dataframe
print('Doing rows: ', end='')
dictL = []

for (iY,tY) in enumerate(all_years):  # a bit hacky to iterate over years and weeks; should iterate over week,year pairs fm 1st to last in file
    for (iW,tW) in enumerate(range(52)):
        for (iC,tC) in enumerate(all_types):
            df0 = dfD[tC] # split by category
            tDatetime = datetime.datetime.strptime('%d-%02d-Sun'%(tY,tW), '%Y-%U-%a')  # need to add weekday for %U to work, see py docs

            issuedByNowIx = df0.issueDatetime <= tDatetime
            expiredByNowIx = df0.expDatetime >= tDatetime
            
            desIx = (issuedByNowIx & ~expiredByNowIx)
            
            # now iterate only over the census blocks found to have records - might be a bit of premature
            # opt, but probably saves some time b/c a quick profile shows sum(desIx) is often zero
            cbV = df0.census_block_2010[desIx]
            cbL = np.unique(cbV)
            for (iB,tB) in enumerate(cbL):
                tV = np.sum(cbV==tB)
            
                # create the new row
                dictL.append({'feature_id': 'business_licenses_in_effect', 
                                'feature_type': tC, 
                                'year': tY, 'week': tW, 
                                'census_block_2010': tB,
                                'value': tV})
        print(len(dictL), end=', ')  # print update after each week


Doing rows: 0, 0, 14, 32, 58, 116, 466, 816, 1167, 1521, 1875, 2342, 2811, 3283, 3960, 4641, 5324, 6008, 6702, 7397, 8094, 8792, 9495, 10205, 10924, 11646, 12509, 13486, 14464, 15442, 16421, 17496, 18574, 19654, 20734, 21815, 22914, 24130, 25346, 26563, 27797, 29115, 30436, 31758, 33093, 34492, 35892, 37292, 38693, 40164, 41635, 43108, 

In [14]:
# make the df from the list of dicts
newdf1 = pd.DataFrame(dictL)
# some diagnostics
print(len(newdf1))
print(newdf1.columns)
newdf1.loc[0:10,:]

43108
Index(['census_block_2010', 'feature_id', 'feature_type', 'value', 'week',
       'year'],
      dtype='object')


,census_block_2010,feature_id,feature_type,value,week,year
0,-30004.0,business_licenses_in_effect,General Business Licenses,1,2,2015
1,-29979.0,business_licenses_in_effect,General Business Licenses,1,2,2015
2,-29974.0,business_licenses_in_effect,General Business Licenses,1,2,2015
3,-29971.0,business_licenses_in_effect,General Business Licenses,1,2,2015
4,-29960.0,business_licenses_in_effect,General Business Licenses,1,2,2015
5,-29959.0,business_licenses_in_effect,General Business Licenses,1,2,2015
6,-30029.0,business_licenses_in_effect,One Family Rental,1,2,2015
7,-29986.0,business_licenses_in_effect,One Family Rental,1,2,2015
8,-29985.0,business_licenses_in_effect,One Family Rental,1,2,2015
9,-29975.0,business_licenses_in_effect,One Family Rental,1,2,2015


In [15]:
# 2nd dataframe with business_licenses_issued_last_4_weeks
print('Doing rows: ', end='')
dictL = []
for (iY,tY) in enumerate(all_years):
    for (iW,tW) in enumerate(range(52)):
        for (iC,tC) in enumerate(all_types):
            df0 = dfD[tC] # split by category
            endDt = datetime.datetime.strptime('%d-%02d-Sun'%(tY,tW), '%Y-%U-%a')  # need to add weekday for %U to work, see py docs
            beginDt = endDt - datetime.timedelta(days=28)
            
            issuedByNowIx = df0.issueDatetime <= tDatetime
            expiredByNowIx = df0.expDatetime >= tDatetime
            desIx = (df0.issueDatetime >= beginDt) & (df0.issueDatetime <= endDt)
            
            # now iterate only over the census blocks found to have records - might be a bit of premature
            # opt, but probably saves some time b/c a quick profile shows sum(desIx) is often zero
            cbV = df0.census_block_2010[desIx]
            cbL = np.unique(cbV)
            for (iB,tB) in enumerate(cbL):
                tV = np.sum(cbV==tB)
            
                # create the new row
                dictL.append({'feature_id': 'business_licenses_issued_last_4_weeks',
                                'feature_type': tC, 
                                'year': tY, 'week': tW, 
                                'census_block_2010': tB,
                                'value': tV})
        print(len(dictL), end=', ')  # print update after each week

Doing rows: 0, 0, 27, 75, 145, 236, 605, 967, 1314, 1670, 2024, 2255, 2502, 2748, 3267, 3682, 4105, 4523, 4949, 5041, 5170, 5302, 5432, 5549, 5671, 5792, 6220, 6826, 7421, 8015, 8396, 8719, 9045, 9369, 9701, 9781, 9859, 10315, 10764, 11226, 11709, 12118, 12555, 13009, 13490, 13939, 14411, 14903, 15426, 16061, 16802, 17617, 

In [16]:
# make the df from the list of dicts
newdf2 = pd.DataFrame(dictL)
# some diagnostics
print(len(newdf2))
print(newdf2.columns)
newdf2.loc[0:10,:]

17617
Index(['census_block_2010', 'feature_id', 'feature_type', 'value', 'week',
       'year'],
      dtype='object')


,census_block_2010,feature_id,feature_type,value,week,year
0,-29976.0,business_licenses_issued_last_4_weeks,Apartment,1,2,2015
1,-29967.0,business_licenses_issued_last_4_weeks,Beauty Shop Nails,1,2,2015
2,-29987.0,business_licenses_issued_last_4_weeks,Delicatessen,1,2,2015
3,-30004.0,business_licenses_issued_last_4_weeks,General Business Licenses,1,2,2015
4,-29988.0,business_licenses_issued_last_4_weeks,General Business Licenses,1,2,2015
5,-29979.0,business_licenses_issued_last_4_weeks,General Business Licenses,1,2,2015
6,-29974.0,business_licenses_issued_last_4_weeks,General Business Licenses,1,2,2015
7,-29971.0,business_licenses_issued_last_4_weeks,General Business Licenses,1,2,2015
8,-29969.0,business_licenses_issued_last_4_weeks,General Business Licenses,1,2,2015
9,-29960.0,business_licenses_issued_last_4_weeks,General Business Licenses,1,2,2015


In [20]:
# write output csv
fulloutdf = pd.concat([newdf1,newdf2])
fulloutdf.to_csv(outName, index=False)